# **식약처 레시피 데이터 메뉴명과 연동**
# **1 농축산식품 레시피 수집기**
## **01 농축산식품 API 및 CSV 전처리**
**[레시피 재료정보 API](http://data.mafra.go.kr/opendata/data/indexOpenDataDetail.do?data_id=20150827000000000465&filter_ty=O&getBack=G&sort_id=&s_data_nm=&instt_id=&cl_code=&shareYn=)**
1. **[glob 모듈에서 파일목록 sorting](https://redcarrot.tistory.com/222)**

In [1]:
ls ./data/레시피*.*

./data/레시피_메뉴정보_20191220.csv  ./data/레시피_조리법_20191220.csv
./data/레시피_재료정보_20191220.csv  ./data/레시피_조리법_원본_20191220.csv


In [2]:
# API 를 활용한 데이터 호출
url = "http://211.237.50.150:7080/openapi/{API_KEY}/json/{OpenAPI 서비스 URL}/{START_INDEX}/{END_INDEX}"
apiKey = 'e5e7753740311d44c7c15c7b5744f5723109b9f5c46eedc47fa7e183ceee22fa'
url = "http://211.237.50.150:7080/openapi/"+apiKey +"/json/Grid_20150827000000000227_1/1/2"
url_sample = "http://211.237.50.150:7080/openapi/sample/json/Grid_20150827000000000227_1/1/2"
from urllib import parse, request
import json
json.loads(request.urlopen(url).read().decode('utf-8'))

{'Grid_20150827000000000227_1': {'totalCnt': 6104,
  'startRow': 1,
  'endRow': 2,
  'result': {'code': 'INFO-000', 'message': '정상 처리되었습니다.'},
  'row': [{'ROW_NUM': 1,
    'RECIPE_ID': 1,
    'IRDNT_SN': 1,
    'IRDNT_NM': '쌀',
    'IRDNT_CPCTY': '4컵',
    'IRDNT_TY_CODE': '3060001',
    'IRDNT_TY_NM': '주재료'},
   {'ROW_NUM': 2,
    'RECIPE_ID': 1,
    'IRDNT_SN': 2,
    'IRDNT_NM': '안심',
    'IRDNT_CPCTY': '200g',
    'IRDNT_TY_CODE': '3060001',
    'IRDNT_TY_NM': '주재료'}]}}

## **02 농축산식품 API 및 CSV 전처리**
CSV 다운파일 중 전처리 작업의 진행

In [3]:
# 조리법 데이터 전처리 재작업 (내부 "," 와 구분자 "," 를 혼용)
with open("data/레시피_조리법_원본_20191220.csv","r", encoding="cp949") as f:
    data = f.readlines()

# 구분자 재정의 하기
import pandas as pd
data = [_.replace('",', '"|')  for _ in data]
data = [_.split("|") for _ in data]
df = pd.DataFrame(data[1:]) # 컬럼을 제외한 데이터만 입력
data[0][0] = data[0][0].replace('\n','')
df.columns = data[0][0].split(",")

# 데이터 내부에 포함된 '"' 지우기
for _ in df.columns:
    df[_] = df[_].fillna('')
    df[_] = list(map(lambda x : x.replace('"',''), df[_]))
df.to_csv("data/레시피_조리법_20191220.csv", sep="|", index=None)
df.head(2)

,레시피 코드,요리설명순서,요리설명,과정 이미지 URL,과정팁
0,195453,5,"모양을 낸 반죽에 밀가루를 묻히고, 계란을 푼 계란물을 묻혀준다.",http://file.okdab.com/recipe/14829957727100013...,\n
1,195453,4,동그랑땡 반죽을 동글납작하게 모양을 잡아준다.,http://file.okdab.com/recipe/14829957727060013...,\n


In [ ]:
# {df_menus_codes[k]:v for k,v in df_tips.items()}
# df_menu_codes[527]

# **2 CSV 다운로드 파일 살피기 및 합치기**
## **01 레시피 다운로드 CSV**
```
['data/레시피_메뉴정보_20191220.csv',
 'data/레시피_재료정보_20191220.csv',
 'data/레시피_조리법_20191220.csv']
```

In [ ]:
# 다운로드 파일을 활용한 자료활용
from glob import glob
food_list = sorted(glob("data/레시피*.*"))
food_list

In [ ]:
import pandas as pd
df_menus = pd.read_csv(food_list[0], encoding='cp949')
df_menus.head(2)

In [4]:
# 레시피 유용한 팁들
# 자연어 검색엔진을 활용 및 적용하기

import pandas as pd
df_menus = pd.read_csv(food_list[0], encoding='cp949')
df_menus_codes = {str(df_menus.loc[_,"레시피 코드"]):df_menus.loc[_,"레시피 이름"]  for _ in range(len(df_menus))}
df_tips = {df['레시피 코드'][_]:df.과정팁[_].strip()  for _ in range(len(df.과정팁)) if len(df.과정팁[_].strip())>3}

NameError: name 'food_list' is not defined

In [ ]:
df_recipe = pd.read_csv(food_list[1], encoding='cp949')
df_recipe.head(2)

In [ ]:
df_cook = pd.read_csv(food_list[2], sep="|")
# df_cook["레시피 코드"] = list(map(lambda x:int(x), df_cook["레시피 코드"]))
# df_cook["요리설명순서"] = list(map(lambda x:int(x), df_cook["요리설명순서"]))
df_cook.head(2)

In [ ]:
df_menus.head(2)

## **02 CSV 파일 합치기**
```
['data/레시피_메뉴정보_20191220.csv',
 'data/레시피_재료정보_20191220.csv',
 'data/레시피_조리법_20191220.csv']
```

In [ ]:
%%time
# 메뉴 수집용 Code 를 dict 으로 추출
df_menus      = pd.read_csv(food_list[0], encoding='cp949')
df_menu_codes = {df_menus.loc[_]["레시피 코드"]:df_menus.loc[_]["레시피 이름"] for _ in range(len(df_menus))}

# 세부 레시피 수집기 작업시작
result = []
for _ in df_menu_codes.keys():
    recipe = df_recipe[df_recipe['레시피 코드'] == _]
    result.append({recipe.loc[_]['재료명']:recipe.loc[_]['재료용량']   for _ in list(recipe.index)})

# 메뉴 컬럼내용 수집 및 전처리 작업의 진행
result = list(map(lambda x: json.dumps(x, ensure_ascii=False), result))
df_menus.insert(3,'상세 레시피', result)
df_menus_columns = ['레시피 이름', "레시피 코드", '상세 레시피', '음식분류코드',\
                    '음식분류','조리시간','분량','대표이미지 URL']
df_menus = df_menus.reindex(columns=df_menus_columns)
# df_menus = df_menus.loc[:, df_menus_columns]
df_menus.to_csv("data/food_recipe_marf.csv", index=None)
df_menus.head(2)